In [1]:
import pandas as pd
import numpy as np
import sqlite3
from tabulate import tabulate

### Functions

In [2]:
def query(cursor, command:str, multiple:bool = True):
    cursor.execute(command)
    result = []
    result.append([description[0] for description in cursor.description])
    #result.append(" | ".join(columns))
    if multiple:
        query_fetch = cursor.fetchall()
        for row in query_fetch:
            result.append(row)
    else:
        query_fetch = cursor.fetchone()
        result.append(query_fetch)
    return result

In [3]:
def print_result(result):
    print(tabulate(result,tablefmt='psql'))

In [4]:
def save_result_to_csv(result, name:str):
    headers = result[0]
    records = result[1:]
    df = pd.DataFrame(records, columns=headers)
    df.to_csv(f"AggregatedData/{name}.csv", index = False)

### Setting up SQL server

In [5]:
df = pd.read_csv("IowaLiquorSales_cleaned.csv")

C:\Users\maksp\AppData\Local\Temp\ipykernel_36960\387395398.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("IowaLiquorSales_cleaned.csv")


In [6]:
conn = sqlite3.connect(':memory:')

In [7]:
df.to_sql('sales', conn, index=False, if_exists='replace')

29260539

In [8]:
cursor = conn.cursor()

In [ ]:
#conn.close()

### Preview of a query result

In [10]:
print_result(query(cursor, "SELECT * FROM sales LIMIT 1"))

+-----------------+------------+--------------+-------------------------+-------------------------+---------+---------+-----------+------------------+---------------+-----------------+-------------+-----------------------+------+------------------+-------------------+---------------------+--------------+--------+--------------------+---------------------+
| invoice         | date       | store_number | store_name              | address                 | city    | county  | category  | category_name    | vendor_number | vendor_name     | item_number | item_description      | pack | bottle_volume_ml | state_bottle_cost | state_bottle_retail | bottles_sold | sale   | volume_sold_liters | volume_sold_gallons |
| INV-27000400013 | 05/04/2020 | 5815         | SUPER STOP IV - DUBUQUE | 535 HILL STREET SUITE A | DUBUQUE | DUBUQUE | 1011100.0 | BLENDED WHISKIES | 65.0          | JIM BEAM BRANDS | 24454       | KESSLER BLEND WHISKEY | 24   | 375              | 3.15              | 4.73           

## Executing queries

In [12]:
query_result = query(cursor, "SELECT date, SUM(sale) as all_revenue FROM sales GROUP BY date ORDER BY all_revenue DESC")

In [13]:
print_result(query_result)

+------------+--------------------+
| date       | all_revenue        |
| 10/04/2013 | 3504719.75         |
| 10/11/2013 | 3267912.73         |
| 12/22/2020 | 3097698.01         |
| 10/13/2022 | 3088636.83         |
| 10/12/2022 | 2917869.5          |
| 10/05/2021 | 2903928.2          |
| 12/07/2023 | 2743396.67         |
| 12/27/2022 | 2722392.57         |
| 12/29/2020 | 2717979.52         |
| 12/06/2023 | 2694046.76         |
| 08/03/2022 | 2668865.93         |
| 12/07/2022 | 2625570.52         |
| 05/03/2023 | 2625538.98         |
| 08/25/2021 | 2587799.22         |
| 06/29/2022 | 2544467.74         |
| 06/29/2023 | 2543242.34         |
| 12/09/2021 | 2536057.02         |
| 10/06/2021 | 2532396.91         |
| 02/08/2024 | 2468985.37         |
| 03/07/2024 | 2449720.77         |
| 08/31/2022 | 2429289.43         |
| 08/29/2023 | 2421791.18         |
| 09/29/2020 | 2420417.11         |
| 12/05/2022 | 2409943.42         |
| 05/23/2024 | 2399634.38         |
| 12/27/2023 | 2391598.91   

In [14]:
save_result_to_csv(query_result, "date_all_revenue")

## Used queries library

Number of transactions for each county
``` SQL
SELECT county, COUNT(invoice) as n_transactions  FROM sales GROUP BY county ORDER BY n_transactions DESC
```

Number of liters bought for each county
``` SQL
SELECT county, SUM(volume_sold_liters) as n_liters_bought  FROM sales GROUP BY county ORDER BY n_liters_bought DESC
```

Number of transactions for each store
``` SQL
SELECT store_name, COUNT(invoice) as n_transactions  FROM sales GROUP BY store_name ORDER BY n_transactions DESC
```

Number of liters bought for each store
``` SQL
SELECT store_name, SUM(volume_sold_liters) as n_liters_bought  FROM sales GROUP BY store_name ORDER BY n_liters_bought DESC
```

Number of transactions for each city
``` SQL
SELECT city, COUNT(invoice) as n_transactions  FROM sales GROUP BY city ORDER BY n_transactions DESC
```

Number of liters bought for each city
``` SQL
SELECT city, SUM(volume_sold_liters) as n_liters_bought  FROM sales GROUP BY city ORDER BY n_liters_bought DESC
```

Number of transactions for each category
``` SQL
SELECT category_name, COUNT(invoice) as n_transactions  FROM sales GROUP BY category_name ORDER BY n_transactions DESC
```

Number of liters bought for each category
``` SQL
SELECT category_name, SUM(volume_sold_liters) as n_liters_bought  FROM sales GROUP BY category_name ORDER BY n_liters_bought DESC
```

Number of transaction for each vendor
``` SQL
SELECT vendor_name, COUNT(invoice) as n_transactions  FROM sales GROUP BY vendor_name ORDER BY n_transactions DESC
```

Number of liters bought for each vendor
``` SQL
SELECT vendor_name, SUM(volume_sold_liters) as n_liters_bought  FROM sales GROUP BY vendor_name ORDER BY n_liters_bought DESC
```

Number of transactions for each item
``` SQL
SELECT item_description, COUNT(invoice) as n_transactions  FROM sales GROUP BY item_description ORDER BY n_transactions DESC
```

Number of liters bought for each item
``` SQL
SELECT item_description, SUM(volume_sold_liters) as n_liters_bought  FROM sales GROUP BY item_description ORDER BY n_liters_bought DESC
```

Number of transaction for each pack size
``` SQL
SELECT pack, COUNT(invoice) as n_transactions  FROM sales GROUP BY pack ORDER BY n_transactions DESC
```

Number of transactions for each category for each county
``` SQL
SELECT county, category_name, COUNT(invoice) as n_transactions  FROM sales GROUP BY county,category_name ORDER BY n_transactions DESC
```

Number of liters bought for each category for each county
``` SQL
SELECT county, category_name, SUM(volume_sold_liters) as n_liters_bought  FROM sales GROUP BY county, category_name ORDER BY n_liters_bought DESC
```

Number of transaction for each category for each city
``` SQL
SELECT city, category_name, COUNT(invoice) as n_transactions  FROM sales GROUP BY city,category_name ORDER BY n_transactions DESC
```

Number of liters bought for each category for each city
``` SQL
SELECT city, category_name, SUM(volume_sold_liters) as n_liters_bought  FROM sales GROUP BY city, category_name ORDER BY n_liters_bought DESC
```

Number of transaction for each item for each county
``` SQL
SELECT county, item_description, COUNT(invoice) as n_transactions  FROM sales GROUP BY county, item_description ORDER BY n_transactions DESC
```

Number of liters bought for each item for each county
``` SQL
SELECT county, item_description, SUM(volume_sold_liters) as n_liters_bought  FROM sales GROUP BY county, item_description ORDER BY n_liters_bought DESC
```

Number of transactions for each item for each city
``` SQL
SELECT city, item_description, COUNT(invoice) as n_transactions  FROM sales GROUP BY city, item_description ORDER BY n_transactions DESC
```

Number of liters bought for each item for each city
``` SQL
SELECT city, item_description, SUM(volume_sold_liters) as n_liters_bought  FROM sales GROUP BY city, item_description ORDER BY n_liters_bought DESC
```

Number of transactions for each day
``` SQL
SELECT date, COUNT(invoice) as n_transactions  FROM sales GROUP BY date ORDER BY n_transactions DESC
```

Number of liters bought for each day
``` SQL
SELECT date, SUM(volume_sold_liters) as n_literes  FROM sales GROUP BY date ORDER BY n_literes DESC
```

Table for number of transactions for each category for each day
``` SQL
CREATE TABLE categories_date as SELECT date, category_name, COUNT(invoice) as n_transactions FROM sales GROUP BY date,category_name ORDER BY date DESC
```

Extracting 1 top category for each day
``` SQL
SELECT date, category_name, MAX(n_transactions) as top_category FROM categories_date GROUP BY date ORDER BY date DESC
```

Revenue for each category
``` SQL
SELECT category_name, SUM(state_bottle_retail) as revenue  FROM sales GROUP BY category_name ORDER BY revenue DESC
```

Table for number of transactions for each category for each day
``` SQL
CREATE TABLE categories_date as SELECT date, category_name, COUNT(invoice) as n_transactions FROM sales GROUP BY date,category_name ORDER BY date DESC
```

Table for number of transactions for each day
``` SQL
CREATE TABLE transactions_date as SELECT date, COUNT(invoice) as n_transactions  FROM sales GROUP BY date ORDER BY n_transactions DESC
```

Number of transactions for American vodka and all alcohols for each day
``` SQL
SELECT categories_date.date, categories_date.n_transactions as USA_vodka, transactions_date.n_transactions as all FROM categories_date inner join transactions_date on categories_date.date=transactions_date.date WHERE categories_date.category_name = 'AMERICAN VODKAS'
```

Number of American Vodkas transaction for each county
``` SQL
SELECT county, COUNT(invoice) as n_transactions  FROM sales WHERE category_name = 'AMERICAN VODKAS' GROUP BY county
```

Table for number of American Vodka transactions for each store
``` SQL
CREATE TABLE store_county_vodka as SELECT store, COUNT(invoice) as vodka_n_transactions FROM sales WHERE category_name = 'AMERICAN VODKAS' GROUP BY store ORDER BY vodka_n_transcations DESC
```

Final result for number of transactions of american vodka and all categories for each store
``` SQL
SELECT sales.store_name, COUNT(sales.invoice) as n_transactions, store_county_vodka.vodka_n_transactions FROM sales JOIN store_county_vodka ON sales.store_name = store_county_vodka.store_name GROUP BY sales.store_name ORDER BY n_transactions DESC
```

Number of transactions for american vodka and all for each store
``` SQL
SELECT sales.store_name, COUNT(sales.invoice) as n_transactions, store_county_vodka.vodka_n_transactions FROM sales JOIN store_county_vodka ON sales.store_name = store_county_vodka.store_name GROUP BY sales.store_name ORDER BY n_transactions DESC
```

Revenue for American Vodkas for each county for each day
``` SQL
SELECT date, county, SUM(sale) as revenue FROM sales WHERE category_name = 'AMERICAN VODKAS' GROUP BY date, county ORDER BY revenue DESC
```